# An introduction to Memtable in Ibis 

In [4]:
import ibis
import pandas as pd
import numpy as np

## Working with pandas dataframes in Ibis 

read in data from February 2009 from the New York City Taxi data set

In [5]:
df = pd.read_csv('yellow_tripdata_2009-02.csv')

In [6]:
df.head()

,vendor_name,Trip_Pickup_DateTime,Trip_Dropoff_DateTime,Passenger_Count,Trip_Distance,Start_Lon,Start_Lat,Rate_Code,store_and_forward,End_Lon,End_Lat,Payment_Type,Fare_Amt,surcharge,mta_tax,Tip_Amt,Tolls_Amt,Total_Amt
0,DDS,2009-02-03 08:25:00,2009-02-03 08:33:39,1.0,1.60,-73.992768,40.758325,NaN,NaN,-73.994710,40.739723,CASH,6.9,0.0,NaN,0.0,0.0,6.9
1,VTS,2009-02-28 00:26:00,2009-02-28 00:40:00,5.0,3.05,0.000000,0.000000,NaN,NaN,0.000000,0.000000,CASH,10.5,0.5,NaN,0.0,0.0,11.0
2,DDS,2009-02-22 00:39:23,2009-02-22 00:45:52,1.0,1.50,-73.137393,41.366138,NaN,NaN,-73.137393,41.366138,CASH,5.7,0.5,NaN,0.0,0.0,6.2
3,VTS,2009-02-28 12:47:00,2009-02-28 12:54:00,1.0,1.42,-73.980458,40.748445,NaN,NaN,-73.996103,40.737095,CASH,6.1,0.0,NaN,0.0,0.0,6.1
4,DDS,2009-02-05 18:34:35,2009-02-05 18:43:26,1.0,1.50,-73.137393,41.366138,NaN,NaN,-73.137393,41.366138,CASH,6.9,1.0,NaN,0.0,0.0,7.9


Set interactive mode to false to be able to see underlying details

In [7]:
ibis.options.interactive = False

Call memtable! 

In [8]:
ibis.memtable(df)

PandasInMemoryTable
  data:
    DataFrameProxy:
              vendor_name Trip_Pickup_DateTime Trip_Dropoff_DateTime  \
      0               DDS  2009-02-03 08:25:00   2009-02-03 08:33:39   
      1               VTS  2009-02-28 00:26:00   2009-02-28 00:40:00   
      2               DDS  2009-02-22 00:39:23   2009-02-22 00:45:52   
      3               VTS  2009-02-28 12:47:00   2009-02-28 12:54:00   
      4               DDS  2009-02-05 18:34:35   2009-02-05 18:43:26   
      ...             ...                  ...                   ...   
      4723740         VTS  2009-02-25 06:36:00   2009-02-25 06:42:00   
      4723741         VTS  2009-02-25 15:27:00   2009-02-25 15:34:00   
      4723742         VTS  2009-02-22 09:57:00   2009-02-22 10:18:00   
      4723743         VTS  2009-02-24 18:46:00   2009-02-24 19:05:00   
      4723744         VTS                   20                   NaN   

               Passenger_Count  Trip_Distance  Start_Lon  Start_Lat  Rate_Code  \
     

save the table to a variable, t, set interactive mode to true and view column names 

In [9]:
t = ibis.memtable(df)
ibis.options.interactive = True
t.columns

['vendor_name',
 'Trip_Pickup_DateTime',
 'Trip_Dropoff_DateTime',
 'Passenger_Count',
 'Trip_Distance',
 'Start_Lon',
 'Start_Lat',
 'Rate_Code',
 'store_and_forward',
 'End_Lon',
 'End_Lat',
 'Payment_Type',
 'Fare_Amt',
 'surcharge',
 'mta_tax',
 'Tip_Amt',
 'Tolls_Amt',
 'Total_Amt']

count the number of vendors listed 

In [10]:
t.vendor_name.count()

4723745

Examine the SQL code being run by DuckDB when we find the sum of the surcharge 

In [11]:
t.surcharge.sum()

849724.0

In [12]:
ibis.show_sql(t.surcharge.sum())

SELECT
  SUM(t0.surcharge) AS sum
FROM _ibis_memtable1 AS t0


## Performance gains

Create a pandas dataframe with 100000 rows 

In [13]:
df_large = pd.DataFrame({"a": np.random.randn(int(1e8)), "b": ['a','b'] * int(1e8 // 2)}).astype({"b": "string"})

In [14]:
df_large.head()

,a,b
0,0.357165,a
1,0.078334,b
2,0.380782,a
3,-0.531058,b
4,-0.082034,a


Time pandas when calling the value counts operation

In [15]:
%time df_large.b.value_counts()

CPU times: user 1.91 s, sys: 30.6 ms, total: 1.94 s
Wall time: 1.94 s


a    50000000
b    50000000
Name: b, dtype: Int64

Change dataframe to ibis expression

In [16]:
t_large = ibis.memtable(df_large)

Time Ibis when calling the value counts operation

In [17]:
%time t_large.b.value_counts().execute()

CPU times: user 1.83 s, sys: 24.3 ms, total: 1.85 s
Wall time: 275 ms


,b,count
0,a,50000000
1,b,50000000


How much faster is Ibis? 

In [18]:
 1.94*1e3/275

7.054545454545455

Ibis is 7 times faster than pandas!